<a href="https://colab.research.google.com/github/ptobarra/spain-ai-python-course-20201122/blob/main/bloqueIV_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio Bloque IV
A continuación os presento el ejercicio del bloque IV, correspondiente a la introducción a la ventana de comandos, librería argparse y subprocess. 
El siguiente script realiza un modelo de clasificación de uno de los datasets más conocidos para empezar dentro del mundo del Data Science, el dataset iris. Sin entrar en detalle del modelo, este script genera un modelo de clasificación a partir de esos datos muestra la precisión del modelo. Al final del script, se encuentra comentado unas líneas de código de cómo se realizaría una predicción con este modelo. Sin embargo, está fuera del scope de este proyecto y se deja por si queréis ejecutarlas y ver cómo haríamos esta tarea en un proyecto de Data Science.

Muchos de estos modelos, tienen una serie de parámetros por defecto que podemos cambiar para generar una predicción diferente. Lo que se suele hacer para no modificar cada vez que queremos hacer una prueba diferente el script, es añadir parámetros que permitan su ejecución desde la ventana de comandos. Esta va a ser nuestra tarea.

Para ello, identifica primero en el script las siguientes variables: RANDOM_STATE, TEST_SIZE y N_NEIGHBORS.


Lo que vamos a hacer, es utilizar argparse para que podamos modificar estas variables desde la ventana de comandos. 
- RANDOM_STATE: Con valor por defecto None, tiene que ser entero.
- TEST_SIZE: Puede ser valores tipo float de 0 a 1 de 0.1 en 0.1. Por defecto 0.3
- N_NEIGHBORS: Por defecto 3, Tiene que ser entero y mayor que 0.

A posteriori, cuando tengamos creado este script, lo que haremos será generar otro desde el cual llamaremos a este usando la librería subprocess y hacemos un print del output generado por el modelo usando los parámetros por defecto definidos arriba.

In [87]:
%%sh
mkdir -p exercise

In [88]:
%%writefile exercise/model.py
#!/usr/bin/env python

from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics 

#### CONVIERTE LAS SIGUIENTES VARIABLES A ARGUMENTOS DE ARGPARSE
# RANDOM_STATE=1
# TEST_SIZE=0.4
# N_NEIGHBORS=3

import argparse

parser = argparse.ArgumentParser(description="Script that trains and test a machine learning model")

def round_float(value):
    ivalue = round(float(value),1)
    if ((ivalue<0) or (ivalue>1)):
        raise argparse.ArgumentTypeError("%s is not in the range [0,1]" % value)
    return ivalue

def check_positive(value):
    ivalue = int(value)
    if ivalue <= 0:
        raise argparse.ArgumentTypeError("%s is an invalid positive int value" % value)
    return ivalue    

parser.add_argument("--random_state","-random", type= int,
                    help="random_state value int, default=None",
                    default=None)

parser.add_argument("--test_size","-test_size", type= round_float,
                    help="test_size value float in the range [0,1] rounded to 1 digit, default=0.3",
                    default=0.3)

parser.add_argument("--n_neighbors","-n_neighbors", type= check_positive,
                    help="number of neighbours value int > 0, default=3",
                    default=3)

args= parser.parse_args()

RANDOM_STATE=args.random_state
TEST_SIZE=args.test_size
N_NEIGHBORS=args.n_neighbors

# load the iris dataset as an example 
iris = load_iris() 
  
# store the feature matrix (X) and response vector (y) 
X = iris.data 
y = iris.target 
  
# splitting X and y into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE) 
  
# training the model on training set 
knn = KNeighborsClassifier(n_neighbors=N_NEIGHBORS) 
knn.fit(X_train, y_train) 
  
# making predictions on the testing set 
y_pred = knn.predict(X_test) 
  
# comparing actual response values (y_test) with predicted response values (y_pred) 
print("model accuracy:", metrics.accuracy_score(y_test, y_pred)) 


# making prediction for out of sample data 
sample = [[3, 5, 4, 2], [2, 3, 5, 4]] 
preds = knn.predict(sample) 
pred_species = [iris.target_names[p] for p in preds] 
print("\nPredictions:", pred_species) 


Overwriting exercise/model.py


In [89]:
%%sh
chmod +x exercise/model.py

In [90]:
%%sh
./exercise/model.py -h

usage: model.py [-h] [--random_state RANDOM_STATE] [--test_size TEST_SIZE]
                [--n_neighbors N_NEIGHBORS]

Script that trains and test a machine learning model

optional arguments:
  -h, --help            show this help message and exit
  --random_state RANDOM_STATE, -random RANDOM_STATE
                        random_state value int, default=None
  --test_size TEST_SIZE, -test_size TEST_SIZE
                        test_size value float in the range [0,1] rounded to 1
                        digit, default=0.3
  --n_neighbors N_NEIGHBORS, -n_neighbors N_NEIGHBORS
                        number of neighbours value int > 0, default=3


In [91]:
%%sh
python exercise/model.py # -random 1234 -test_size 0.4 -n_neighbors 4

model accuracy: 1.0

Predictions: ['versicolor', 'virginica']


In [92]:
%%sh
python exercise/model.py -random 1234 -test_size 0.4 -n_neighbors 4

model accuracy: 0.9666666666666667

Predictions: ['versicolor', 'virginica']


In [93]:
%%sh
python exercise/model.py -random 1234 -test_size 1.15 -n_neighbors 4

usage: model.py [-h] [--random_state RANDOM_STATE] [--test_size TEST_SIZE]
                [--n_neighbors N_NEIGHBORS]
model.py: error: argument --test_size/-test_size: 1.15 is not in the range [0,1]


In [94]:
%%sh
python exercise/model.py -random 1234 -test_size 0.4 -n_neighbors -1

usage: model.py [-h] [--random_state RANDOM_STATE] [--test_size TEST_SIZE]
                [--n_neighbors N_NEIGHBORS]
model.py: error: argument --n_neighbors/-n_neighbors: -1 is an invalid positive int value


En la segunda parte, usa subprocess para llamar a este script generado

In [152]:
%%writefile exercise/call_model.py
#!/usr/bin/env python

import subprocess

### EJECUTA EL script del modelo y haz un print del output generado con los argumentos por defecto 

completed = subprocess.run(['python', 'exercise/model.py', '-random', '1234', '-test_size', '0.4', '-n_neighbors', '4'], stdout=subprocess.PIPE)

lines = str(completed.stdout).split("\\n")
line_1 = lines[0][2:]
line_2 = lines[2]

print(str(completed.stdout))
print()
print(lines)
print('\n' + line_1)
print('\n' + line_2)

Overwriting exercise/call_model.py


In [153]:
%%sh
chmod +x exercise/call_model.py

In [154]:
%%sh
./exercise/call_model.py

b"model accuracy: 0.9666666666666667\n\nPredictions: ['versicolor', 'virginica']\n"

['b"model accuracy: 0.9666666666666667', '', "Predictions: ['versicolor', 'virginica']", '"']

model accuracy: 0.9666666666666667

Predictions: ['versicolor', 'virginica']
